# Import

In [1]:
package_paths = [
    'FMix-master'
]
import sys; 

for pth in package_paths:
    sys.path.append(pth)
    
from fmix import sample_mask, make_low_freq_image, binarise_mask

In [2]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import timm

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
#from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

warnings.filterwarnings("ignore")

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

# Config

In [4]:
CFG = {
    'fold_num': 5,
    'seed': 719,
    'model_arch': 'tf_efficientnet_b4_ns',
    'img_size': 720,
    'epochs': 2,
    'train_bs': 8,
    'valid_bs': 16,
    'T_0': 10,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 4,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:1'
}

# Data Preparation

In [5]:
# video_labels = pd.read_csv('/home/thinh/nfl/train_labels.csv').fillna(0)
# video_labels = video_labels[video_labels['frame'] != 0].reset_index(drop=True)
# video_labels['video_name'] = video_labels['video'].apply(lambda x: "_".join(x.split("_")[:2]))
# video_valid = ['57583_000082', '57586_004152', '57911_000147', '57997_003691', '57680_002206', '58095_004022', '57906_000718', '58005_001254', '57679_003316', '58103_003494', '57998_002181', '58048_000086']
# video_labels = video_labels[~video_labels.video_name.isin(video_valid)]

# video_labels_with_impact = video_labels[video_labels['impact'] > 0]
# for row in tqdm(video_labels_with_impact[['video','frame','label']].values):
#     frames = np.array([-4,-3,-2,-1,1,2,3,4])+row[1]
#     video_labels.loc[(video_labels['video'] == row[0]) 
#                                  & (video_labels['frame'].isin(frames))
#                                  & (video_labels['label'] == row[2]), 'impact'] = 1


# video_labels['image_name'] = video_labels['video'].str.replace('.mp4', '') + '_' + video_labels['frame'].astype(str) + '.png'
# video_labels.to_csv('frame_train_labels.csv', index=False)

In [6]:
video_labels = pd.read_csv('frame_train_labels.csv')
train = video_labels.groupby('image_name')['impact'].sum().reset_index()
train['impact'] = train['impact'].apply(lambda x: 1 if x > 0 else 0)
train.columns = ['image_id', 'label']
train.head()

,image_id,label
0,57584_000336_Endzone_1.png,0
1,57584_000336_Endzone_10.png,0
2,57584_000336_Endzone_100.png,0
3,57584_000336_Endzone_101.png,0
4,57584_000336_Endzone_102.png,0


In [7]:
image_id_label_0 = train[train['label'] == 0].image_id.tolist()
image_id_label_1 = train[train['label'] == 1].image_id.tolist()
image_id_label_0 = random.sample(image_id_label_0, 10000)
image_ids = image_id_label_0 + image_id_label_1

train = train[train.image_id.isin(image_ids)].reset_index(drop=True)

In [8]:
train.label.value_counts()

0    10000
1     8004
Name: label, dtype: int64

In [9]:
train['group'] = train['image_id'].apply(lambda x: "_".join(x.split('_')[:2]))

In [10]:
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

# Albumentations

In [11]:
def get_train_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
  
        
def get_valid_transforms():
    return Compose([
            CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

# Dataset

In [12]:
def rand_bbox(size, lam):
    W = size[0]
    H = size[1]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2


class CassavaDataset(Dataset):
    def __init__(self, df, data_root, 
                 transforms=None, 
                 output_label=True, 
                 one_hot_label=False,
                 do_fmix=False, 
                 fmix_params={
                     'alpha': 1., 
                     'decay_power': 3., 
                     'shape': (CFG['img_size'], CFG['img_size']),
                     'max_soft': True, 
                     'reformulate': False
                 },
                 do_cutmix=False,
                 cutmix_params={
                     'alpha': 1,
                 }
                ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.do_fmix = do_fmix
        self.fmix_params = fmix_params
        self.do_cutmix = do_cutmix
        self.cutmix_params = cutmix_params
        
        self.output_label = output_label
        self.one_hot_label = one_hot_label
        
        if output_label == True:
            self.labels = self.df['label'].values
            #print(self.labels)
            
            if one_hot_label is True:
                self.labels = np.eye(self.df['label'].max()+1)[self.labels]
                #print(self.labels)
            
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.labels[index]
          
        img = get_img("{}/{}".format(self.data_root, self.df.loc[index]['image_id']))

        if self.transforms:
            img = self.transforms(image=img)['image']
        
        if self.do_fmix and np.random.uniform(0., 1., size=1)[0] > 0.5:
            with torch.no_grad():
                #lam, mask = sample_mask(**self.fmix_params)
                
                lam = np.clip(np.random.beta(self.fmix_params['alpha'], self.fmix_params['alpha']),0.6,0.7)
                
                # Make mask, get mean / std
                mask = make_low_freq_image(self.fmix_params['decay_power'], self.fmix_params['shape'])
                mask = binarise_mask(mask, lam, self.fmix_params['shape'], self.fmix_params['max_soft'])
    
                fmix_ix = np.random.choice(self.df.index, size=1)[0]
                fmix_img  = get_img("{}/{}".format(self.data_root, self.df.iloc[fmix_ix]['image_id']))

                if self.transforms:
                    fmix_img = self.transforms(image=fmix_img)['image']

                mask_torch = torch.from_numpy(mask)
                
                # mix image
                img = mask_torch*img+(1.-mask_torch)*fmix_img

                #print(mask.shape)

                #assert self.output_label==True and self.one_hot_label==True

                # mix target
                rate = mask.sum()/CFG['img_size']/CFG['img_size']
                target = rate*target + (1.-rate)*self.labels[fmix_ix]
                #print(target, mask, img)
                #assert False
        
        if self.do_cutmix and np.random.uniform(0., 1., size=1)[0] > 0.5:
            #print(img.sum(), img.shape)
            with torch.no_grad():
                cmix_ix = np.random.choice(self.df.index, size=1)[0]
                cmix_img  = get_img("{}/{}".format(self.data_root, self.df.iloc[cmix_ix]['image_id']))
                if self.transforms:
                    cmix_img = self.transforms(image=cmix_img)['image']
                    
                lam = np.clip(np.random.beta(self.cutmix_params['alpha'], self.cutmix_params['alpha']),0.3,0.4)
                bbx1, bby1, bbx2, bby2 = rand_bbox((CFG['img_size'], CFG['img_size']), lam)

                img[:, bbx1:bbx2, bby1:bby2] = cmix_img[:, bbx1:bbx2, bby1:bby2]

                rate = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (CFG['img_size'] * CFG['img_size']))
                target = rate*target + (1.-rate)*self.labels[cmix_ix]
                
            #print('-', img.sum())
            #print(target)
            #assert False
                            
        # do label smoothing
        #print(type(img), type(target))
        if self.output_label == True:
            return img, target
        else:
            return img

# Model

In [13]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        '''
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            #nn.Linear(n_features, hidden_size,bias=True), nn.ELU(),
            nn.Linear(n_features, n_class, bias=True)
        )
        '''
    def forward(self, x):
        x = self.model(x)
        return x

# Training APIs

In [14]:
def prepare_dataloader(df, trn_idx, val_idx, data_root='/home/thinh/nfl/train_images/'):
    
#     from catalyst.data.sampler import BalanceClassSampler
    
    train_ = df.loc[trn_idx,:].reset_index(drop=True)
    valid_ = df.loc[val_idx,:].reset_index(drop=True)
        
    train_ds = CassavaDataset(train_, 
                              data_root, 
                              transforms=get_train_transforms(), 
                              output_label=True, 
                              one_hot_label=False, 
                              do_fmix=False, 
                              do_cutmix=False)
    valid_ds = CassavaDataset(valid_, 
                              data_root, 
                              transforms=get_valid_transforms(), 
                              output_label=True)
    
    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=CFG['train_bs'],
        pin_memory=False,
        drop_last=False,
        shuffle=True,        
        num_workers=CFG['num_workers'],
        #sampler=BalanceClassSampler(labels=train_['label'].values, mode="downsampling")
    )
    val_loader = torch.utils.data.DataLoader(
        valid_ds, 
        batch_size=CFG['valid_bs'],
        num_workers=CFG['num_workers'],
        shuffle=False,
        pin_memory=False,
    )
    return train_loader, val_loader

def train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, device, scheduler=None, schd_batch_update=False):
    model.train()

    t = time.time()
    running_loss = None

    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()

        #print(image_labels.shape, exam_label.shape)
        with autocast():
            image_preds = model(imgs)   #output = model(input)
            #print(image_preds.shape, exam_pred.shape)

            loss = loss_fn(image_preds, image_labels)
            
            scaler.scale(loss).backward()

            if running_loss is None:
                running_loss = loss.item()
            else:
                running_loss = running_loss * .99 + loss.item() * .01

            if ((step + 1) %  CFG['accum_iter'] == 0) or ((step + 1) == len(train_loader)):
                # may unscale_ here if desired (e.g., to allow clipping unscaled gradients)

                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad() 
                
                if scheduler is not None and schd_batch_update:
                    scheduler.step()

            if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(train_loader)):
                description = f'epoch {epoch} loss: {running_loss:.4f}'
                
                pbar.set_description(description)
                
    if scheduler is not None and not schd_batch_update:
        scheduler.step()
        
def valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False):
    model.eval()

    t = time.time()
    loss_sum = 0
    sample_num = 0
    image_preds_all = []
    image_targets_all = []
    
    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()
        
        image_preds = model(imgs)   #output = model(input)
        #print(image_preds.shape, exam_pred.shape)
        image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
        image_targets_all += [image_labels.detach().cpu().numpy()]
        
        loss = loss_fn(image_preds, image_labels)
        
        loss_sum += loss.item()*image_labels.shape[0]
        sample_num += image_labels.shape[0]  

        if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(val_loader)):
            description = f'epoch {epoch} loss: {loss_sum/sample_num:.4f}'
            pbar.set_description(description)
    
    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    print(np.unique(image_preds_all, return_counts=True))
    print(np.unique(image_targets_all, return_counts=True))
    print('validation multi-class accuracy = {:.4f}'.format((image_preds_all==image_targets_all).mean()))
    print('recall = {:.4f}'.format(recall_score(image_targets_all, image_preds_all)))
    print('precision = {:.4f}'.format(precision_score(image_targets_all, image_preds_all)))
    
    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum/sample_num)
        else:
            scheduler.step()

In [15]:
# reference: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/173733
class MyCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean'):
        super().__init__(weight=weight, reduction=reduction)
        self.weight = weight
        self.reduction = reduction

    def forward(self, inputs, targets):
        lsm = F.log_softmax(inputs, -1)

        if self.weight is not None:
            lsm = lsm * self.weight.unsqueeze(0)

        loss = -(targets * lsm).sum(-1)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

# Main Loop

In [16]:
if __name__ == '__main__':
     # for training only, need nightly build pytorch

    seed_everything(CFG['seed'])
    
    folds = GroupKFold(n_splits=CFG['fold_num']).split(np.arange(train.shape[0]), train.label.values, train.group.values)
    
    for fold, (trn_idx, val_idx) in enumerate(folds):
        # we'll train fold 0 first
#         if fold > 0:
#             break 

        print('Training with {} started'.format(fold))

        print(len(trn_idx), len(val_idx))
        train_loader, val_loader = prepare_dataloader(train, trn_idx, val_idx, data_root='/home/thinh/nfl/train_images/')

        device = torch.device(CFG['device'])
        
        model = CassvaImgClassifier(CFG['model_arch'], train.label.nunique(), pretrained=True).to(device)
        scaler = GradScaler()   
        optimizer = torch.optim.Adam(model.parameters(), lr=CFG['lr'], weight_decay=CFG['weight_decay'])
        #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=CFG['epochs']-1)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=CFG['T_0'], T_mult=1, eta_min=CFG['min_lr'], last_epoch=-1)
        #scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=25, 
        #                                                max_lr=CFG['lr'], epochs=CFG['epochs'], steps_per_epoch=len(train_loader))
        
        loss_tr = nn.CrossEntropyLoss().to(device) #MyCrossEntropyLoss().to(device)
        loss_fn = nn.CrossEntropyLoss().to(device)
        
        for epoch in range(CFG['epochs']):
            train_one_epoch(epoch, model, loss_tr, optimizer, train_loader, device, scheduler=scheduler, schd_batch_update=False)

            with torch.no_grad():
                valid_one_epoch(epoch, model, loss_fn, val_loader, device, scheduler=None, schd_loss_update=False)

            torch.save(model.state_dict(),'/home/thinh/nfl/frame-models/{}_1024_fold_{}_{}'.format(CFG['model_arch'], fold, epoch))
            
        #torch.save(model.cnn_model.state_dict(),'{}/cnn_model_fold_{}_{}'.format(CFG['model_path'], fold, CFG['tag']))
        
        del model, optimizer, train_loader, val_loader, scaler, scheduler
        with torch.cuda.device(CFG['device']):
            torch.cuda.empty_cache()

Training with 0 started
14563 3441


epoch 0 loss: 0.6178: 100%|█████████▉| 1820/1821 [48:47<00:01,  1.61s/it]


RuntimeError: CUDA out of memory. Tried to allocate 3.64 GiB (GPU 1; 10.92 GiB total capacity; 3.25 GiB already allocated; 3.64 GiB free; 6.60 GiB reserved in total by PyTorch)

In [ ]:
# Training with 0 started
# 2777 695
# epoch 0 loss: 0.6624: 100%|██████████| 174/174 [04:38<00:00,  1.60s/it]
# epoch 0 loss: 0.6424: 100%|██████████| 22/22 [00:13<00:00,  1.63it/s]
# (array([0, 1]), array([487, 208]))
# (array([0, 1]), array([400, 295]))
# validation multi-class accuracy = 0.5669
# epoch 1 loss: 0.6275: 100%|██████████| 174/174 [04:44<00:00,  1.63s/it]
# epoch 1 loss: 0.6427: 100%|██████████| 22/22 [00:12<00:00,  1.70it/s]
# (array([0, 1]), array([391, 304]))
# (array([0, 1]), array([400, 295]))
# validation multi-class accuracy = 0.6158
# epoch 2 loss: 0.6086: 100%|██████████| 174/174 [04:46<00:00,  1.65s/it]
# epoch 2 loss: 0.6374: 100%|██████████| 22/22 [00:11<00:00,  1.90it/s]
# (array([0, 1]), array([506, 189]))
# (array([0, 1]), array([400, 295]))
# validation multi-class accuracy = 0.6288
# epoch 3 loss: 0.6281: 100%|██████████| 174/174 [04:42<00:00,  1.62s/it]
# epoch 3 loss: 0.6487: 100%|██████████| 22/22 [00:11<00:00,  1.92it/s]
# (array([0, 1]), array([498, 197]))
# (array([0, 1]), array([400, 295]))
# validation multi-class accuracy = 0.6374
# epoch 4 loss: 0.5527: 100%|██████████| 174/174 [04:44<00:00,  1.64s/it]
# epoch 4 loss: 0.6521: 100%|██████████| 22/22 [00:11<00:00,  1.89it/s]
# (array([0, 1]), array([337, 358]))
# (array([0, 1]), array([400, 295]))
# validation multi-class accuracy = 0.6475
# epoch 5 loss: 0.5262: 100%|██████████| 174/174 [04:45<00:00,  1.64s/it]
# epoch 5 loss: 0.6504: 100%|██████████| 22/22 [00:11<00:00,  1.91it/s]
# (array([0, 1]), array([339, 356]))
# (array([0, 1]), array([400, 295]))
# validation multi-class accuracy = 0.6417
# epoch 6 loss: 0.5045: 100%|██████████| 174/174 [04:45<00:00,  1.64s/it]
# epoch 6 loss: 0.6450: 100%|██████████| 22/22 [00:11<00:00,  1.92it/s]
# (array([0, 1]), array([337, 358]))
# (array([0, 1]), array([400, 295]))
# validation multi-class accuracy = 0.6388
# epoch 7 loss: 0.5159: 100%|██████████| 174/174 [04:46<00:00,  1.64s/it]
# epoch 7 loss: 0.6613: 100%|██████████| 22/22 [00:11<00:00,  1.86it/s]
# (array([0, 1]), array([414, 281]))
# (array([0, 1]), array([400, 295]))
# validation multi-class accuracy = 0.6403
# epoch 8 loss: 0.5050: 100%|██████████| 174/174 [04:46<00:00,  1.65s/it]
# epoch 8 loss: 0.6582: 100%|██████████| 22/22 [00:12<00:00,  1.69it/s]
# (array([0, 1]), array([400, 295]))
# (array([0, 1]), array([400, 295]))
# validation multi-class accuracy = 0.6460
# epoch 9 loss: 0.4887: 100%|██████████| 174/174 [04:46<00:00,  1.65s/it]
# epoch 9 loss: 0.6603: 100%|██████████| 22/22 [00:12<00:00,  1.69it/s]
# (array([0, 1]), array([419, 276]))
# (array([0, 1]), array([400, 295]))
# validation multi-class accuracy = 0.6446


# Training with 0 started
# 16013 4004
# epoch 0 loss: 0.6003: 100%|██████████| 1001/1001 [26:28<00:00,  1.59s/it]
# epoch 0 loss: 0.5525: 100%|██████████| 126/126 [01:02<00:00,  2.01it/s]
# (array([0, 1]), array([1346, 2658]))
# (array([0, 1]), array([2000, 2004]))
# validation multi-class accuracy = 0.7223
# epoch 1 loss: 0.5661: 100%|██████████| 1001/1001 [26:34<00:00,  1.59s/it]
# epoch 1 loss: 0.5421: 100%|██████████| 126/126 [01:01<00:00,  2.06it/s]
# (array([0, 1]), array([1379, 2625]))
# (array([0, 1]), array([2000, 2004]))
# validation multi-class accuracy = 0.7365
# epoch 2 loss: 0.5376: 100%|██████████| 1001/1001 [26:35<00:00,  1.59s/it]
# epoch 2 loss: 0.5190: 100%|██████████| 126/126 [01:01<00:00,  2.06it/s]
# (array([0, 1]), array([1406, 2598]))
# (array([0, 1]), array([2000, 2004]))
# validation multi-class accuracy = 0.7393


# Training with 0 started
# 15999 4018
# epoch 0 loss: 0.5194: 100%|██████████| 1000/1000 [26:29<00:00,  1.59s/it]
# epoch 0 loss: 0.6937: 100%|██████████| 126/126 [01:01<00:00,  2.06it/s]
# (array([0, 1]), array([1546, 2472]))
# (array([0, 1]), array([1953, 2065]))
# validation multi-class accuracy = 0.6493
# acc = 0.6493
# recall = 0.7574
# precision = 0.6327
# epoch 1 loss: 0.4363: 100%|██████████| 1000/1000 [26:32<00:00,  1.59s/it]
# epoch 1 loss: 0.8429: 100%|██████████| 126/126 [00:59<00:00,  2.13it/s]
# (array([0, 1]), array([2741, 1277]))
# (array([0, 1]), array([1953, 2065]))
# validation multi-class accuracy = 0.6063
# acc = 0.6063
# recall = 0.4262
# precision = 0.6891
# epoch 2 loss: 0.3699: 100%|██████████| 1000/1000 [26:31<00:00,  1.59s/it]
# epoch 2 loss: 0.8512: 100%|██████████| 126/126 [00:59<00:00,  2.13it/s]
# (array([0, 1]), array([2135, 1883]))
# (array([0, 1]), array([1953, 2065]))
# validation multi-class accuracy = 0.6237
# acc = 0.6237
# recall = 0.5898
# precision = 0.6468
# epoch 3 loss: 0.3070: 100%|██████████| 1000/1000 [26:30<00:00,  1.59s/it]
# epoch 3 loss: 0.9448: 100%|██████████| 126/126 [00:59<00:00,  2.13it/s]
# (array([0, 1]), array([2261, 1757]))
# (array([0, 1]), array([1953, 2065]))
# validation multi-class accuracy = 0.6401
# acc = 0.6401
# recall = 0.5753
# precision = 0.6762
# epoch 4 loss: 0.2391: 100%|██████████| 1000/1000 [26:30<00:00,  1.59s/it]
# epoch 4 loss: 1.2499: 100%|██████████| 126/126 [00:59<00:00,  2.13it/s]
# (array([0, 1]), array([2371, 1647]))
# (array([0, 1]), array([1953, 2065]))
# validation multi-class accuracy = 0.6297
# acc = 0.6297
# recall = 0.5385
# precision = 0.6752
# epoch 5 loss: 0.1974: 100%|██████████| 1000/1000 [26:29<00:00,  1.59s/it]
# epoch 5 loss: 1.4107: 100%|██████████| 126/126 [00:59<00:00,  2.13it/s]
# (array([0, 1]), array([2810, 1208]))
# (array([0, 1]), array([1953, 2065]))
# validation multi-class accuracy = 0.5941
# acc = 0.5941
# recall = 0.3976
# precision = 0.6796
# epoch 6 loss: 0.1483: 100%|██████████| 1000/1000 [26:29<00:00,  1.59s/it]
# epoch 6 loss: 1.4747: 100%|██████████| 126/126 [00:59<00:00,  2.13it/s]
# (array([0, 1]), array([2737, 1281]))
# (array([0, 1]), array([1953, 2065]))
# validation multi-class accuracy = 0.5978
# acc = 0.5978
# recall = 0.4189
# precision = 0.6753
# epoch 7 loss: 0.1342: 100%|██████████| 1000/1000 [26:29<00:00,  1.59s/it]
# epoch 7 loss: 1.6187: 100%|██████████| 126/126 [00:59<00:00,  2.13it/s]
# (array([0, 1]), array([2729, 1289]))
# (array([0, 1]), array([1953, 2065]))
# validation multi-class accuracy = 0.6023
# acc = 0.6023
# recall = 0.4252
# precision = 0.6811


# Training with 0 started
# 16001 4016
# epoch 0 loss: 0.5603: 100%|██████████| 1001/1001 [26:31<00:00,  1.59s/it]
# epoch 0 loss: 0.6357: 100%|██████████| 126/126 [01:01<00:00,  2.06it/s]
# (array([0, 1]), array([2207, 1809]))
# (array([0, 1]), array([1974, 2042]))
# validation multi-class accuracy = 0.6536
# acc = 0.6536
# recall = 0.6024
# precision = 0.6799
# epoch 1 loss: 0.5130: 100%|██████████| 1001/1001 [26:33<00:00,  1.59s/it]
# epoch 1 loss: 0.7619: 100%|██████████| 126/126 [00:59<00:00,  2.11it/s]
# (array([0, 1]), array([2612, 1404]))
# (array([0, 1]), array([1974, 2042]))
# validation multi-class accuracy = 0.6350
# acc = 0.6350
# recall = 0.4848
# precision = 0.7051
# epoch 2 loss: 0.4182: 100%|██████████| 1001/1001 [27:01<00:00,  1.62s/it]
# epoch 2 loss: 0.8542: 100%|██████████| 126/126 [01:01<00:00,  2.05it/s]
# (array([0, 1]), array([2494, 1522]))
# (array([0, 1]), array([1974, 2042]))
# validation multi-class accuracy = 0.6434
# acc = 0.6434
# recall = 0.5220
# precision = 0.7004
# epoch 3 loss: 0.3825: 100%|██████████| 1001/1001 [27:07<00:00,  1.63s/it]
# epoch 3 loss: 0.9308: 100%|██████████| 126/126 [01:00<00:00,  2.09it/s]
# (array([0, 1]), array([2302, 1714]))
# (array([0, 1]), array([1974, 2042]))
# validation multi-class accuracy = 0.6409
# acc = 0.6409
# recall = 0.5666
# precision = 0.6750




# Training with 0 started
# 15996 4021
# epoch 0 loss: 0.4292: 100%|██████████| 1000/1000 [26:58<00:00,  1.62s/it]
# epoch 0 loss: 0.6624: 100%|██████████| 126/126 [01:02<00:00,  2.01it/s]
# (array([0, 1]), array([2140, 1881]))
# (array([0, 1]), array([2047, 1974]))
# validation multi-class accuracy = 0.6481
# recall = 0.6180
# precision = 0.6486
# epoch 1 loss: 0.2773: 100%|██████████| 1000/1000 [27:14<00:00,  1.63s/it]
# epoch 1 loss: 0.8931: 100%|██████████| 126/126 [01:00<00:00,  2.09it/s]
# (array([0, 1]), array([2835, 1186]))
# (array([0, 1]), array([2047, 1974]))
# validation multi-class accuracy = 0.5951
# recall = 0.3880
# precision = 0.6459






# Training with 0 started
# 16005 4012
# epoch 0 loss: 0.5832: 100%|██████████| 1001/1001 [27:16<00:00,  1.63s/it]
# epoch 0 loss: 0.6006: 100%|██████████| 126/126 [01:05<00:00,  1.92it/s]
# (array([0, 1]), array([1140, 2872]))
# (array([0, 1]), array([2037, 1975]))
# validation multi-class accuracy = 0.6852
# recall = 0.9073
# precision = 0.6240
# epoch 1 loss: 0.5595: 100%|██████████| 1001/1001 [27:22<00:00,  1.64s/it]
# epoch 1 loss: 0.5865: 100%|██████████| 126/126 [01:04<00:00,  1.95it/s]
# (array([0, 1]), array([1668, 2344]))
# (array([0, 1]), array([2037, 1975]))
# validation multi-class accuracy = 0.6837
# recall = 0.7722
# precision = 0.6506
# epoch 2 loss: 0.5249: 100%|██████████| 1001/1001 [27:23<00:00,  1.64s/it]
# epoch 2 loss: 0.5920: 100%|██████████| 126/126 [01:04<00:00,  1.96it/s]
# (array([0, 1]), array([1589, 2423]))
# (array([0, 1]), array([2037, 1975]))
# validation multi-class accuracy = 0.6825
# recall = 0.7909
# precision = 0.6447
# epoch 3 loss: 0.4903: 100%|██████████| 1001/1001 [27:24<00:00,  1.64s/it]
# epoch 3 loss: 0.5893: 100%|██████████| 126/126 [01:04<00:00,  1.95it/s]
# (array([0, 1]), array([1830, 2182]))
# (array([0, 1]), array([2037, 1975]))
# validation multi-class accuracy = 0.7006
# recall = 0.7484
# precision = 0.6774
# epoch 4 loss: 0.4799: 100%|██████████| 1001/1001 [27:26<00:00,  1.64s/it]
# epoch 4 loss: 0.6795: 100%|██████████| 126/126 [01:05<00:00,  1.93it/s]
# (array([0, 1]), array([1605, 2407]))
# (array([0, 1]), array([2037, 1975]))
# validation multi-class accuracy = 0.6934
# recall = 0.7980
# precision = 0.6548
# epoch 5 loss: 0.4500: 100%|██████████| 1001/1001 [27:18<00:00,  1.64s/it]
# epoch 5 loss: 0.6719: 100%|██████████| 126/126 [01:03<00:00,  1.98it/s]
# (array([0, 1]), array([1842, 2170]))
# (array([0, 1]), array([2037, 1975]))
# validation multi-class accuracy = 0.6897
# recall = 0.7342
# precision = 0.6682
# epoch 6 loss: 0.4287: 100%|██████████| 1001/1001 [27:21<00:00,  1.64s/it]
# epoch 6 loss: 0.6843: 100%|██████████| 126/126 [01:03<00:00,  1.98it/s]
# (array([0, 1]), array([2006, 2006]))
# (array([0, 1]), array([2037, 1975]))
# validation multi-class accuracy = 0.6907
# recall = 0.6937
# precision = 0.6830
# epoch 7 loss: 0.4077: 100%|██████████| 1001/1001 [27:20<00:00,  1.64s/it]
# epoch 7 loss: 0.7017: 100%|██████████| 126/126 [01:03<00:00,  1.98it/s]
# (array([0, 1]), array([1846, 2166]))
# (array([0, 1]), array([2037, 1975]))
# validation multi-class accuracy = 0.6947
# recall = 0.7382
# precision = 0.6731
# epoch 8 loss: 0.3889: 100%|██████████| 1001/1001 [27:19<00:00,  1.64s/it]
# epoch 8 loss: 0.7242: 100%|██████████| 126/126 [01:05<00:00,  1.93it/s]
# (array([0, 1]), array([1918, 2094]))
# (array([0, 1]), array([2037, 1975]))
# validation multi-class accuracy = 0.6937
# recall = 0.7190
# precision = 0.6781
# epoch 9 loss: 0.3869: 100%|██████████| 1001/1001 [27:16<00:00,  1.64s/it]
# epoch 9 loss: 0.7539: 100%|██████████| 126/126 [01:04<00:00,  1.97it/s]
# (array([0, 1]), array([2176, 1836]))
# (array([0, 1]), array([2037, 1975]))
# validation multi-class accuracy = 0.6837
# recall = 0.6435
# precision = 0.6923




# Training with 0 started
# 16003 4014
# epoch 0 loss: 0.5466: 100%|██████████| 1001/1001 [27:09<00:00,  1.63s/it]
# epoch 0 loss: 0.6560: 100%|██████████| 126/126 [01:06<00:00,  1.89it/s]
# (array([0, 1]), array([1868, 2146]))
# (array([0, 1]), array([2047, 1967]))
# validation multi-class accuracy = 0.6425
# recall = 0.6807
# precision = 0.6240
# epoch 1 loss: 0.4770: 100%|██████████| 1001/1001 [27:25<00:00,  1.64s/it]
# epoch 1 loss: 0.6956: 100%|██████████| 126/126 [01:11<00:00,  1.76it/s]
# (array([0, 1]), array([2083, 1931]))
# (array([0, 1]), array([2047, 1967]))
# validation multi-class accuracy = 0.6587
# recall = 0.6426
# precision = 0.6546


#################################### EfficientNet-B4

# Training with 0 started
# 15992 4025
# epoch 0 loss: 0.6061: 100%|██████████| 1000/1000 [27:14<00:00,  1.63s/it]
# epoch 0 loss: 0.5896: 100%|██████████| 126/126 [01:06<00:00,  1.91it/s]
# (array([0, 1]), array([1317, 2708]))
# (array([0, 1]), array([2178, 1847]))
# validation multi-class accuracy = 0.6678
# recall = 0.8711
# precision = 0.5942
# epoch 1 loss: 0.5720: 100%|██████████| 1000/1000 [27:26<00:00,  1.65s/it]
# epoch 1 loss: 0.5771: 100%|██████████| 126/126 [01:05<00:00,  1.92it/s]
# (array([0, 1]), array([1828, 2197]))
# (array([0, 1]), array([2178, 1847]))
# validation multi-class accuracy = 0.6989
# recall = 0.7666
# precision = 0.6445
# epoch 2 loss: 0.5468: 100%|██████████| 1000/1000 [27:26<00:00,  1.65s/it]
# epoch 2 loss: 0.5787: 100%|██████████| 126/126 [01:04<00:00,  1.97it/s]
# (array([0, 1]), array([1512, 2513]))
# (array([0, 1]), array([2178, 1847]))
# validation multi-class accuracy = 0.6860
# recall = 0.8381
# precision = 0.6160
# epoch 3 loss: 0.5159: 100%|██████████| 1000/1000 [27:25<00:00,  1.65s/it]
# epoch 3 loss: 0.6714: 100%|██████████| 126/126 [01:03<00:00,  1.98it/s]
# (array([0, 1]), array([2118, 1907]))
# (array([0, 1]), array([2178, 1847]))
# validation multi-class accuracy = 0.6820
# recall = 0.6697
# precision = 0.6487
# epoch 4 loss: 0.5076: 100%|██████████| 1000/1000 [27:27<00:00,  1.65s/it]
# epoch 4 loss: 0.6022: 100%|██████████| 126/126 [01:04<00:00,  1.97it/s]
# (array([0, 1]), array([1568, 2457]))
# (array([0, 1]), array([2178, 1847]))
# validation multi-class accuracy = 0.6681
# recall = 0.8035
# precision = 0.6040
# epoch 5 loss: 0.4765: 100%|██████████| 1000/1000 [27:08<00:00,  1.63s/it]
# epoch 5 loss: 0.6339: 100%|██████████| 126/126 [01:01<00:00,  2.04it/s]
# (array([0, 1]), array([2152, 1873]))
# (array([0, 1]), array([2178, 1847]))
# validation multi-class accuracy = 0.6999
# recall = 0.6800
# precision = 0.6706
# epoch 6 loss: 0.4565: 100%|██████████| 1000/1000 [26:33<00:00,  1.59s/it]
# epoch 6 loss: 0.6661: 100%|██████████| 126/126 [01:02<00:00,  2.03it/s]
# (array([0, 1]), array([2145, 1880]))
# (array([0, 1]), array([2178, 1847]))
# validation multi-class accuracy = 0.6698
# recall = 0.6492
# precision = 0.6378
# epoch 7 loss: 0.4405: 100%|██████████| 1000/1000 [27:04<00:00,  1.62s/it]
# epoch 7 loss: 0.6883: 100%|██████████| 126/126 [01:03<00:00,  1.97it/s]
# (array([0, 1]), array([2228, 1797]))
# (array([0, 1]), array([2178, 1847]))
# validation multi-class accuracy = 0.6800
# recall = 0.6378
# precision = 0.6555
# epoch 8 loss: 0.4142: 100%|██████████| 1000/1000 [27:23<00:00,  1.64s/it]
# epoch 8 loss: 0.7311: 100%|██████████| 126/126 [01:04<00:00,  1.96it/s]
# (array([0, 1]), array([2404, 1621]))
# (array([0, 1]), array([2178, 1847]))
# validation multi-class accuracy = 0.6706
# recall = 0.5799
# precision = 0.6607



#################################### EfficientNet-B6
# Training with 0 started
# 15999 4018
# epoch 0 loss: 0.6169: 100%|██████████| 2000/2000 [50:08<00:00,  1.50s/it]
# epoch 0 loss: 0.5906: 100%|██████████| 252/252 [01:54<00:00,  2.21it/s]
# (array([0, 1]), array([1424, 2594]))
# (array([0, 1]), array([2056, 1962]))
# validation multi-class accuracy = 0.6909
# recall = 0.8445
# precision = 0.6388
# epoch 1 loss: 0.5837: 100%|██████████| 2000/2000 [50:17<00:00,  1.51s/it]
# epoch 1 loss: 0.6239: 100%|██████████| 252/252 [01:51<00:00,  2.25it/s]
# (array([0, 1]), array([1624, 2394]))
# (array([0, 1]), array([2056, 1962]))
# validation multi-class accuracy = 0.6789
# recall = 0.7813
# precision = 0.6404
# epoch 2 loss: 0.5587: 100%|██████████| 2000/2000 [50:13<00:00,  1.51s/it]
# epoch 2 loss: 0.6101: 100%|██████████| 252/252 [01:51<00:00,  2.25it/s]
# (array([0, 1]), array([1475, 2543]))
# (array([0, 1]), array([2056, 1962]))
# validation multi-class accuracy = 0.6782
# recall = 0.8186
# precision = 0.6315
# Training with 1 started
# 16028 3989
# epoch 0 loss: 0.6161: 100%|██████████| 2004/2004 [49:33<00:00,  1.48s/it]
# epoch 0 loss: 0.6110: 100%|██████████| 250/250 [01:50<00:00,  2.26it/s]
# (array([0, 1]), array([1691, 2298]))
# (array([0, 1]), array([1950, 2039]))
# validation multi-class accuracy = 0.6658
# recall = 0.7366
# precision = 0.6536
# epoch 1 loss: 0.5981: 100%|██████████| 2004/2004 [49:19<00:00,  1.48s/it]
# epoch 1 loss: 0.6524: 100%|██████████| 250/250 [01:49<00:00,  2.29it/s]
# (array([0, 1]), array([2348, 1641]))
# (array([0, 1]), array([1950, 2039]))
# validation multi-class accuracy = 0.6450
# recall = 0.5552
# precision = 0.6898
# epoch 2 loss: 0.5705: 100%|██████████| 2004/2004 [49:18<00:00,  1.48s/it]
# epoch 2 loss: 0.8050: 100%|██████████| 250/250 [01:49<00:00,  2.29it/s]
# (array([0, 1]), array([2660, 1329]))
# (array([0, 1]), array([1950, 2039]))
# validation multi-class accuracy = 0.6340
# recall = 0.4679
# precision = 0.7178
# Training with 2 started
# 16047 3970
# epoch 0 loss: 0.6025: 100%|██████████| 2006/2006 [49:21<00:00,  1.48s/it]
# epoch 0 loss: 0.7158: 100%|██████████| 249/249 [01:48<00:00,  2.29it/s]
# (array([0, 1]), array([1974, 1996]))
# (array([0, 1]), array([1940, 2030]))
# validation multi-class accuracy = 0.6599
# recall = 0.6591
# precision = 0.6703
# epoch 1 loss: 0.5745: 100%|██████████| 2006/2006 [49:23<00:00,  1.48s/it]
# epoch 1 loss: 0.6577: 100%|██████████| 249/249 [01:48<00:00,  2.29it/s]
# (array([0, 1]), array([1582, 2388]))
# (array([0, 1]), array([1940, 2030]))
# validation multi-class accuracy = 0.6680
# recall = 0.7635
# precision = 0.6491




